### Add composite variabels (SI, OSI, OI) to I/O dataframe for Banner
* Query or load the dataframes that are required to to calculate the composite variables
* Calculate the composite variabels
* Add he composite variables to the original I/O dataframe

In [1]:
import numpy as np
import pandas as pd
import pickle
import os

pd.options.display.max_rows = 1000

fileDir = os.path.dirname('__file__')

In [2]:
def queryDF4composites(encounter_ids, hr_df=None, nsbp_df=None,
                       map_df=None, fio2_df=None, spo2_df=None,
                       pao2_df=None):
    
    if hr_df is None:
        hr_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_hr_df.pkl'))    
    hr_df = hr_df.loc[np.in1d(hr_df.ENCNTR_ID, encounter_ids),:]
    
    if nsbp_df is None:
        nsbp_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_nsbp_df.pkl'))
    nsbp_df = nsbp_df.loc[np.in1d(nsbp_df.ENCNTR_ID, encounter_ids), :]
    
    if map_df is None:
        map_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_map_df.pkl'))
    map_df = map_df.loc[np.in1d(map_df.ENCNTR_ID, encounter_ids), :]
    
    if fio2_df is None:
        fio2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_fio2_df.pkl'))
    fio2_df = fio2_df.loc[np.in1d(fio2_df.ENCNTR_ID, encounter_ids), :]
    
    if spo2_df is None:
        spo2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_spo2_df.pkl'))
    spo2_df = spo2_df.loc[np.in1d(spo2_df.ENCNTR_ID, encounter_ids),:]
    
    if pao2_df is None:
        pao2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_pao2_df.pkl'))
    pao2_df = pao2_df.loc[np.in1d(pao2_df.ENCNTR_ID, encounter_ids), :]
    
    hr_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
    nsbp_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
    map_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
    fio2_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
    spo2_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
    pao2_df.sort_values(by=['ENCNTR_ID', 'EVENT_END_DT_TM'], inplace=True)
        
    return hr_df, nsbp_df, map_df, fio2_df, spo2_df, pao2_df

def filtObservation(item_df, scr_df, timelag, timewin):
    scr_df.rename(columns={'ENCNTR_ID': 'encounter_id'}, inplace=True)
    enc_reft = scr_df.groupby('encounter_id')['reftime'].unique().to_frame()
    enc_reft = enc_reft.reset_index()
    enc_reft['reftime'] = np.hstack(enc_reft.reftime)
    
#     item_df = item_df.merge(scr_df.loc[:, ['encounter_id', 'reftime']], on='encounter_id', how='inner')
    item_df.rename(columns={'ENCNTR_ID': 'encounter_id', 
                            'EVENT_END_DT_TM': 'charttime'}, inplace=True)
    item_df = item_df.merge(enc_reft, on='encounter_id', how='inner')
    if timelag > 0:
        item_df['fromtime'] = item_df.reftime
        item_df['totime'] = item_df.reftime + np.timedelta64(int(timelag * 60), 'm')
        # time_mask = (_df.charttime > _df.reftime) & (_df.charttime < _df.totime)
    else:
        item_df['fromtime'] = item_df.reftime + np.timedelta64(int(timelag * 60), 'm')
        item_df['totime'] = item_df.reftime + np.timedelta64(int((timelag + timewin) * 60), 'm')
    time_mask = (item_df.charttime > item_df.fromtime) & (item_df.charttime < item_df.totime)
    item_df = item_df.loc[time_mask, :]
    
    return item_df

In [3]:
uomconv = pickle.load(open(os.path.join(fileDir, 
                                        'pickle_files_banner', 
                                        'banner2ism_uomconv.pkl'), 'rb'))

In [4]:
uomconv

{'albumin': {'banner': 'g/dl', 'conversion': 1, 'ism': 'g/dl'},
 'creatinine': {'banner': 'mg/dl', 'conversion': 1, 'ism': 'mg/dl'},
 'fio2': {'banner': '%', 'conversion': 0.01, 'ism': 'fraction'},
 'glucose': {'banner': 'mg/dl', 'conversion': 1, 'ism': 'mg/dl'},
 'hemoglobin': {'banner': 'g/dl', 'conversion': 1, 'ism': 'g/dl'},
 'hr': {'banner': 'bpm', 'conversion': 1, 'ism': 'bpm'},
 'lactic_acid': {'banner': 'mmol/l', 'conversion': 9.009, 'ism': 'mg/dl'},
 'map': {'banner': 'mmH2O', 'conversion': 0.1, 'ism': 'cmH2O'},
 'ndbp': {'banner': 'mmHg', 'conversion': 1, 'ism': 'mmHg'},
 'nsbp': {'banner': 'mmHg', 'conversion': 1, 'ism': 'mmHg'},
 'pao2': {'banner': 'cmH2O', 'conversion': 0.735541, 'ism': 'mmHg'},
 'ph': {'banner': None, 'conversion': 1, 'ism': None},
 'platelet': {'banner': 'K/ul', 'conversion': 1, 'ism': 'K/ul'},
 'potassium': {'banner': 'meq/l', 'conversion': 1, 'ism': 'meq/l'},
 'ratio_pao2_flo2': {'banner': None, 'conversion': 1, 'ism': None},
 'spo2': {'banner': '%', '

In [5]:
def convertMapUOM(map_df):
    """
    Convert map uom 
    from:mmH2O 
    to:cmH2O
    """
    map_df.RESULT_VAL_num = map_df.RESULT_VAL_num*0.1
    return map_df

def convertFio2UOM(fio2_df):
    """
    Convert FiO2 uom
    from: %
    to: fraction
    """
    fio2_df.RESULT_VAL_num = fio2_df.RESULT_VAL_num*0.01
    return fio2_df

def convertPao2UOM(pao2_df):
    """
    Convert PaO2 uom
    from: cmH2O
    to: mmHg
    """
    pao2_df.RESULT_VAL_num = pao2_df.RESULT_VAL_num*0.735541
    return pao2_df

In [6]:
# si_df_filtered = hr_df_filtered.groupby('encounter_id').apply(getSI, nsbp_df_filtered)
# si_df_filtered = si_df_filtered.loc[:, ['encounter_id', 'charttime', 'reftime', 'fromtime', 'totime', 'si']]
# si_df_filtered.rename(columns={'si':'value'}, inplace=True)
# si_df_filtered['valueUOM'] = 'bpm/mmHg'

def getSI(hr_df_group, nsbp_df_filtered):
    nsbp_df_group = nsbp_df_filtered.loc[nsbp_df_filtered.encounter_id
                                         ==hr_df_group.encounter_id.unique()[0], :]
    hr_df_group['si'] = np.nan    
    tdiff = np.timedelta64(60, 'm')
    if not nsbp_df_group.empty:
        for idx, row in hr_df_group.iterrows():
            idxmin = abs(nsbp_df_group.charttime-row.charttime).idxmin()            
            valmin = abs(nsbp_df_group.charttime-row.charttime).min()
            
            if valmin<=np.timedelta64(60,'m'):
                try:
                    hr_df_group.si[idx] = row.RESULT_VAL_num/float(nsbp_df_group.RESULT_VAL_num[idxmin])
                except:
                    pass
    
    return hr_df_group

def getSIDF(hr_df, nsbp_df):
    si_df = hr_df.groupby('encounter_id').apply(getSI, nsbp_df)
    si_df = si_df.loc[:, ['encounter_id', 'charttime', 'reftime', 'fromtime', 'totime', 'si']]
    si_df.rename(columns={'si':'value'}, inplace=True)
    si_df['valueUOM'] = 'bpm/mmHg'
    si_df = si_df.replace([np.inf, -np.inf], np.nan)
    return si_df
    
def getOSI(spo2_df_group, map_df_filtered, fio2_df_filtered):
    map_df_group = map_df_filtered.loc[map_df_filtered.encounter_id
                                       ==spo2_df_group.encounter_id.unique()[0], :]
    fio2_df_group = fio2_df_filtered.loc[fio2_df_filtered.encounter_id
                                         ==spo2_df_group.encounter_id.unique()[0], :]
    spo2_df_group['osi'] = np.nan
    tdiff = np.timedelta64(60, 'm')
    if not (map_df_group.empty or fio2_df_group.empty):
        for idx, row in spo2_df_group.iterrows():
            idxmin_map = abs(map_df_group.charttime-row.charttime).idxmin()
            valmin_map = abs(map_df_group.charttime-row.charttime).min()
            idxmin_fio2 = abs(fio2_df_group.charttime-row.charttime).idxmin()
            valmin_fio2 = abs(fio2_df_group.charttime-row.charttime).min()
            
            if (valmin_map<=np.timedelta64(60,'m')) and (valmin_fio2<=np.timedelta64(60,'m')):
                try:
                    spo2_df_group.osi[idx] = (map_df_group.RESULT_VAL_num[idxmin_map]
                                              *fio2_df_group.RESULT_VAL_num[idxmin_fio2]*100)/float(row.RESULT_VAL_num)
                except:
                    pass
    
    return spo2_df_group

def getOSIDF(spo2_df, map_df, fio2_df):
    osi_df = spo2_df.groupby('encounter_id').apply(getOSI, map_df, fio2_df)
    osi_df = osi_df.loc[:, ['encounter_id', 'charttime', 'reftime', 'fromtime', 'totime', 'osi']]
    osi_df.rename(columns={'osi':'value'}, inplace=True)
    osi_df['valueUOM'] = 'cmH2O'
    osi_df = osi_df.replace([np.inf, -np.inf], np.nan)
    return osi_df

def getOI(pao2_df_group, map_df_filtered, fio2_df_filtered):
    map_df_group = map_df_filtered.loc[map_df_filtered.encounter_id
                                       ==pao2_df_group.encounter_id.unique()[0], :]
    fio2_df_group = fio2_df_filtered.loc[fio2_df_filtered.encounter_id
                                         ==pao2_df_group.encounter_id.unique()[0], :]
    pao2_df_group['oi'] = np.nan
    tdiff = np.timedelta64(60, 'm')
    if not (map_df_group.empty or fio2_df_group.empty):
        for idx, row in pao2_df_group.iterrows():
            idxmin_map = abs(map_df_group.charttime-row.charttime).idxmin()
            valmin_map = abs(map_df_group.charttime-row.charttime).min()
            idxmin_fio2 = abs(fio2_df_group.charttime-row.charttime).idxmin()
            valmin_fio2 = abs(fio2_df_group.charttime-row.charttime).min()
            
            if (valmin_map<=np.timedelta64(60,'m')) and (valmin_fio2<=np.timedelta64(60,'m')):
                try:
                    pao2_df_group.oi[idx] = (map_df_group.RESULT_VAL_num[idxmin_map]
                                             *fio2_df_group.RESULT_VAL_num[idxmin_fio2]*100)/(row.RESULT_VAL_num*1.36)
                except:
                    pass
    
    return pao2_df_group

def getOIDF(pao2_df, map_df, fio2_df):
    oi_df = pao2_df.groupby('encounter_id').apply(getOI, map_df, fio2_df)
    oi_df = oi_df.loc[:, ['encounter_id', 'charttime', 'reftime', 'fromtime', 'totime', 'oi']]
    oi_df.rename(columns={'oi':'value'}, inplace=True)
    oi_df['valueUOM'] = '%'
    oi_df = oi_df.replace([np.inf, -np.inf], np.nan)
    return oi_df

In [7]:
def getLastValDF(group):
    
    tmp_group = group.loc[~pd.isnull(group.value),:]
    if not tmp_group.empty:
        return tmp_group.value[tmp_group.charttime.idxmax()]
    else:
        return np.NaN
    
def add2IO(io_df, feature_dfs, stats):
    feature_stats = [(feature, stat) for feature in feature_dfs for stat in stats]
    for feature, stat in feature_stats:
        feature_df = feature_dfs[feature]
        ft_name = feature+'_'+stat
#         print(ft_name)
        if stat !='last':
            ft_stat_df = eval("feature_df.groupby('encounter_id')['value']."+stat+"().reset_index()")
        else:
            ft_stat_sr = feature_df.groupby('encounter_id').apply(getLastValDF)
            ft_stat_df = pd.DataFrame(ft_stat_sr)
            ft_stat_df.rename(columns={0: 'value'}, inplace=True)
            ft_stat_df = ft_stat_df.reset_index()
            
        io_df = io_df.merge(ft_stat_df, on='encounter_id', how='left')
        io_df.rename(columns={'value': ft_name}, inplace=True)
    return io_df

In [8]:
fileDir = os.path.dirname("__file__")
timelag = np.arange(-6,-25,-1)
timewin = [6, 12]
stabletime = list([12])
combination = [(tlag, twin, stime) for tlag in timelag for twin in timewin 
               for stime in stabletime if abs(tlag)>=twin]

In [9]:
if not os.path.exists(os.path.join(fileDir, 'io_baner2')):
        os.makedirs(os.path.join(fileDir, 'io_banner2'))

hr_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_hr_df.pkl'))
nsbp_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_nsbp_df.pkl'))
map_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_map_df.pkl'))
fio2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_fio2_df.pkl'))
spo2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_spo2_df.pkl'))
pao2_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_pao2_df.pkl'))


for tlag, twin, stime in combination:
    print('timelag: {}, timewindow: {}'.format(tlag, twin))
    
    fname_scr_df = os.path.join(fileDir, 'scr_banner', 
                                'banner_onset_scr_tlag{:03d}_stime{:03d}_tot.pkl'.format(abs(tlag), stime))
    fname_io_df_aki = os.path.join(fileDir, 'io_banner', 
                                   'banner_onset_io_tlag{:03d}_twin{:03d}_aki.pkl'.format(abs(tlag), twin))
    fname_io_df_con = os.path.join(fileDir, 'io_banner', 
                                   'banner_onset_io_tlag{:03d}_twin{:03d}_con.pkl'.format(abs(tlag), twin))
    
    # Load Creatinine dataframe
    scr_df = pd.read_pickle(fname_scr_df)
        
    # Load I/O dataframe
    io_df_aki = pd.read_pickle(fname_io_df_aki)
    io_df_con = pd.read_pickle(fname_io_df_con)
    
    # Query dataframes to calculate the composite predictors
    encounter_ids_aki = io_df_aki.encounter_id.unique()
    encounter_ids_con = io_df_con.encounter_id.unique()
    
    (hr_df_aki, nsbp_df_aki, map_df_aki, 
     fio2_df_aki, spo2_df_aki, pao2_df_aki) = queryDF4composites(encounter_ids_aki,
                                                                 hr_df=hr_df, nsbp_df=nsbp_df,
                                                                 map_df=map_df, fio2_df=fio2_df, 
                                                                 spo2_df=spo2_df, pao2_df=pao2_df)
    (hr_df_con, nsbp_df_con, map_df_con, 
     fio2_df_con, spo2_df_con, pao2_df_con) = queryDF4composites(encounter_ids_con,
                                                                 hr_df=hr_df, nsbp_df=nsbp_df, 
                                                                 map_df=map_df, fio2_df=fio2_df, 
                                                                 spo2_df=spo2_df, pao2_df=pao2_df)
    print('Queried the dataframes ...')
    
    # Filter dataframes with the observation window
    hr_df_filtered_aki = filtObservation(hr_df_aki, scr_df, tlag, twin)
    nsbp_df_filtered_aki = filtObservation(nsbp_df_aki, scr_df, tlag, twin)
    map_df_filtered_aki = filtObservation(map_df_aki, scr_df, tlag, twin)
    fio2_df_filtered_aki = filtObservation(fio2_df_aki, scr_df, tlag, twin)
    spo2_df_filtered_aki = filtObservation(spo2_df_aki, scr_df, tlag, twin)
    pao2_df_filtered_aki = filtObservation(pao2_df_aki, scr_df, tlag, twin)
    
    hr_df_filtered_con = filtObservation(hr_df_con, scr_df, tlag, twin)
    nsbp_df_filtered_con = filtObservation(nsbp_df_con, scr_df, tlag, twin)
    map_df_filtered_con = filtObservation(map_df_con, scr_df, tlag, twin)
    fio2_df_filtered_con = filtObservation(fio2_df_con, scr_df, tlag, twin)
    spo2_df_filtered_con = filtObservation(spo2_df_con, scr_df, tlag, twin)
    pao2_df_filtered_con = filtObservation(pao2_df_con, scr_df, tlag, twin)
    print('Required dataframes filtered by observation window ...')
    
    # Convert UOM to be consistent with ISM
    map_df_filtered_aki = convertMapUOM(map_df_filtered_aki)
    map_df_filtered_con = convertMapUOM(map_df_filtered_con)
    
    fio2_df_filtered_aki = convertFio2UOM(fio2_df_filtered_aki)
    fio2_df_filtered_con = convertFio2UOM(fio2_df_filtered_con)
    
    pao2_df_filtered_aki = convertPao2UOM(pao2_df_filtered_aki)
    pao2_df_filtered_con = convertPao2UOM(pao2_df_filtered_con)
    print('UOM converted ...')
    
    # Get Shock index (SI) dataframe
    si_df_filtered_aki = getSIDF(hr_df_filtered_aki, nsbp_df_filtered_aki)
    si_df_filtered_con = getSIDF(hr_df_filtered_con, nsbp_df_filtered_con)
    print('Got Shock Index ...')
    
    # Get Oxygenation Saturation Index (OSI) dataframe
    osi_df_filtered_aki = getOSIDF(spo2_df_filtered_aki, map_df_filtered_aki, 
                                   fio2_df_filtered_aki)
    osi_df_filtered_con = getOSIDF(spo2_df_filtered_con, map_df_filtered_con, 
                                   fio2_df_filtered_con)
    print('Got Oxygenation Saturation Index ...')
    
    # Get Oxygenation Index (OI) dataframe
    oi_df_filtered_aki = getOIDF(pao2_df_filtered_aki, map_df_filtered_aki,
                                 fio2_df_filtered_aki)
    oi_df_filtered_con = getOIDF(pao2_df_filtered_con, map_df_filtered_con,
                                 fio2_df_filtered_con)
    print('Got Oxygenation Index ...')
    
    # Add feature statistics to the I/O dataframe
    feature_dfs_aki = {'si': si_df_filtered_aki, 'osi': osi_df_filtered_aki, 'oi': oi_df_filtered_aki}
    feature_dfs_con = {'si': si_df_filtered_con, 'osi': osi_df_filtered_con, 'oi': oi_df_filtered_con}
    stats = ['min', 'max', 'mean', 'last', 'median']
    
    io_df_full_aki = add2IO(io_df_aki, feature_dfs_aki, stats)
    io_df_full_con = add2IO(io_df_con, feature_dfs_con, stats)
    print('Added to the I/O dataframe')    
    
    fname_io_full_aki2 = os.path.join(fileDir, 'io_banner2', 
                                      'banner_onset_io_tlag{:03d}_twin{:03d}_aki.pkl'.format(abs(tlag), twin))
    fname_io_full_con2 = os.path.join(fileDir, 'io_banner2', 
                                      'banner_onset_io_tlag{:03d}_twin{:03d}_con.pkl'.format(abs(tlag), twin))
    io_df_full_aki.to_pickle(fname_io_full_aki2)
    io_df_full_con.to_pickle(fname_io_full_con2)
    
    print('timelag: {}, timewindow: {} finished!! \n'.format(tlag, twin))
    

timelag: -6, timewindow: 6


C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packa

Queried the dataframes ...


C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


Required dataframes filtered by observation window ...
UOM converted ...
Got Shock Index ...
Got Oxygenation Saturation Index ...
Got Oxygenation Index ...
Added to the I/O dataframe
timelag: -6, timewindow: 6 finished!! 

timelag: -7, timewindow: 6
Queried the dataframes ...
Required dataframes filtered by observation window ...
UOM converted ...
Got Shock Index ...
Got Oxygenation Saturation Index ...
Got Oxygenation Index ...
Added to the I/O dataframe
timelag: -7, timewindow: 6 finished!! 

timelag: -8, timewindow: 6
Queried the dataframes ...
Required dataframes filtered by observation window ...
UOM converted ...
Got Shock Index ...
Got Oxygenation Saturation Index ...
Got Oxygenation Index ...
Added to the I/O dataframe
timelag: -8, timewindow: 6 finished!! 

timelag: -9, timewindow: 6
Queried the dataframes ...
Required dataframes filtered by observation window ...
UOM converted ...
Got Shock Index ...
Got Oxygenation Saturation Index ...
Got Oxygenation Index ...
Added to the 

In [54]:
tlag = -6
twin = 6
stime = 12
fname_scr_df = os.path.join(fileDir, 'scr_banner', 
                            'banner_onset_scr_tlag{:03d}_stime{:03d}_tot.pkl'.format(abs(tlag), stime))
fname_io_df_aki = os.path.join(fileDir, 'io_banner2', 
                               'banner_onset_io_tlag{:03d}_twin{:03d}_aki.pkl'.format(abs(tlag), twin))
fname_io_df_con = os.path.join(fileDir, 'io_banner2', 
                               'banner_onset_io_tlag{:03d}_twin{:03d}_con.pkl'.format(abs(tlag), twin))

# Load Creatinine dataframe
scr_df = pd.read_pickle(fname_scr_df)

# Load I/O dataframe
io_df_aki = pd.read_pickle(fname_io_df_aki)
io_df_con = pd.read_pickle(fname_io_df_con)

# Query dataframes to calculate the composite predictors
encounter_ids_aki = io_df_aki.encounter_id.unique()
encounter_ids_con = io_df_con.encounter_id.unique()

(hr_df_aki, nsbp_df_aki, map_df_aki, 
 fio2_df_aki, spo2_df_aki, pao2_df_aki) = queryDF4composites(encounter_ids_aki,
                                                             hr_df=hr_df, nsbp_df=nsbp_df,
                                                             map_df=map_df, fio2_df=fio2_df, 
                                                             spo2_df=spo2_df, pao2_df=pao2_df)
(hr_df_con, nsbp_df_con, map_df_con, 
 fio2_df_con, spo2_df_con, pao2_df_con) = queryDF4composites(encounter_ids_con,
                                                             hr_df=hr_df, nsbp_df=nsbp_df, 
                                                             map_df=map_df, fio2_df=fio2_df, 
                                                             spo2_df=spo2_df, pao2_df=pao2_df)

hr_df_filtered_aki = filtObservation(hr_df_aki, scr_df, tlag, twin)
nsbp_df_filtered_aki = filtObservation(nsbp_df_aki, scr_df, tlag, twin)
map_df_filtered_aki = filtObservation(map_df_aki, scr_df, tlag, twin)
fio2_df_filtered_aki = filtObservation(fio2_df_aki, scr_df, tlag, twin)
spo2_df_filtered_aki = filtObservation(spo2_df_aki, scr_df, tlag, twin)
pao2_df_filtered_aki = filtObservation(pao2_df_aki, scr_df, tlag, twin)

hr_df_filtered_con = filtObservation(hr_df_con, scr_df, tlag, twin)
nsbp_df_filtered_con = filtObservation(nsbp_df_con, scr_df, tlag, twin)
map_df_filtered_con = filtObservation(map_df_con, scr_df, tlag, twin)
fio2_df_filtered_con = filtObservation(fio2_df_con, scr_df, tlag, twin)
spo2_df_filtered_con = filtObservation(spo2_df_con, scr_df, tlag, twin)
pao2_df_filtered_con = filtObservation(pao2_df_con, scr_df, tlag, twin)

si_df_filtered_aki = getSIDF(hr_df_filtered_aki, nsbp_df_filtered_aki)
si_df_filtered_con = getSIDF(hr_df_filtered_con, nsbp_df_filtered_con)

C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packa

in here
in here


NameError: global name 'stop' is not defined

In [55]:
%debug

> <ipython-input-53-d716a89f7468>(19)getSI()
     17 #                 try:
     18                     print('in here')
---> 19                     stop
     20                     hr_df_group.si[idx] = row.value/float(nsbp_df_group.value[idxmin])
     21 #                 except:

ipdb> row.value
*** AttributeError: 'Series' object has no attribute 'value'
ipdb> row
EVENT_CD                                                     3049471
RESULT_UNITS_CD                                                  239
encounter_id       00b719a07ab022ad6b57b87d79e6ca603914258e457a26...
MRN                                                      P23052086.0
EVENT_ID           a44e08bc394576756693d58a41ab3a6f621f7ca0831edf...
EVENT_NAME                                                Heart Rate
EVENT_MEASURE                                             Heart Rate
charttime                                        2041-01-19 02:00:00
ORDER_ID                                                         NaN


In [33]:
hr_df_aki.loc[hr_df_aki.ENCNTR_ID=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68', :]

,EVENT_CD,RESULT_UNITS_CD,ENCNTR_ID,MRN,EVENT_ID,EVENT_NAME,EVENT_MEASURE,EVENT_END_DT_TM,ORDER_ID,RESULT_VAL_num
951,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,6f6633394c7d25d6f4a28caf6de3e6ffcbfc8ccf585ea6...,Heart Rate,Heart Rate,2041-01-15 18:18:00,NaN,148.0
940,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,deb103c5ff1c1dcd7e5495966931ea39015fa35ed07c6a...,Heart Rate,Heart Rate,2041-01-15 18:32:00,NaN,137.0
938,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,f9762eae2b1fe402479f6f50b8c3a14b8b25babd6e9af8...,Heart Rate,Heart Rate,2041-01-15 18:49:00,NaN,136.0
953,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,c3d1d7896550272035b908cea214af579c260807a5442a...,Heart Rate,Heart Rate,2041-01-15 19:40:00,NaN,128.0
944,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,b031f912f89ca5ed4adf17d721d66563fd0b8e95baf0f3...,Heart Rate,Heart Rate,2041-01-15 21:04:00,NaN,117.0
950,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,189ff6d700eda1bbf22ebabc5d610c8fab3e5159bb5423...,Heart Rate,Heart Rate,2041-01-15 23:00:00,NaN,108.0
937,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,575cc219892638a1bb9d4440915cc17d5dde7a017ad451...,Heart Rate,Heart Rate,2041-01-15 23:27:00,NaN,172.0
942,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,805775b13a50e1abdcf52cd80a43300d7b2b50851eda36...,Heart Rate,Heart Rate,2041-01-15 23:27:00,NaN,108.0
947,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,7a7f60e5da1a217f3d35ca26eca0fa1b2d236b031d55bd...,Heart Rate,Heart Rate,2041-01-16 01:08:00,NaN,108.0
952,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,fefb868564faba75cb338ebabbc648b0177341e6f448b8...,Heart Rate,Heart Rate,2041-01-16 03:17:00,NaN,107.0


In [34]:
nsbp_df_aki.loc[nsbp_df_aki.ENCNTR_ID=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68', :]

,EVENT_CD,RESULT_UNITS_CD,ENCNTR_ID,MRN,EVENT_ID,EVENT_NAME,EVENT_MEASURE,EVENT_END_DT_TM,ORDER_ID,RESULT_VAL_num
13289,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,c306958671add99ce2d24fecef53e3db3d8983576e4772...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-15 18:32:00,NaN,107.0
13299,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,f6d745d174c5436b9e0dd86dfc3ce0c19066096b038786...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-15 21:04:00,NaN,104.0
13287,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,a38676eb4f2838ad340c6dbd074652e4be55b478bbfe51...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-15 23:00:00,NaN,106.0
13327,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,4940bfbb38e724765baf9ba8d60c197ecdbb55c2ffe882...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-15 23:27:00,NaN,110.0
13288,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,8301e0ae61bdb8e5074cbad54d7464c79c65245a315b27...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 01:08:00,NaN,106.0
13295,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,e8e3956d41db2c608d886dbe21a1549b09d779cd6d07d7...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 03:17:00,NaN,100.0
13290,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,291b0041fe35f35672e501b9e599a2848c3df55cacfcb1...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 04:26:00,NaN,108.0
13284,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,3c1f4518a5adafd8c7fe87c69c82dec0d2c3d148eee5d9...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 06:00:00,NaN,113.0
13300,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,8a013ed0f51849daa762f65e9ee99f939bc1c5f9da5cca...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 08:00:00,NaN,103.0
13298,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,4a184e2cb455df06f8518dddf374c91393d563c285063b...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-16 10:00:00,NaN,110.0


In [27]:
scr_df.reftime[scr_df.ENCNTR_ID=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68'].unique()[0]

numpy.datetime64('2041-01-19T07:00:17.000000000')

In [41]:
hr_df_filtered_aki.loc[hr_df_filtered_aki.encounter_id=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68', :]

,EVENT_CD,RESULT_UNITS_CD,encounter_id,MRN,EVENT_ID,EVENT_NAME,EVENT_MEASURE,charttime,ORDER_ID,RESULT_VAL_num,reftime,fromtime,totime
45,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,a44e08bc394576756693d58a41ab3a6f621f7ca0831edf...,Heart Rate,Heart Rate,2041-01-19 02:00:00,NaN,119.0,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17
46,3049471,239,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,f2ea253149e3acefb47d324fc924f21f218d9698379bc4...,Heart Rate,Heart Rate,2041-01-19 06:00:00,NaN,134.0,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17


In [43]:
nsbp_df_filtered_aki.loc[nsbp_df_filtered_aki.encounter_id=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68', :]

,EVENT_CD,RESULT_UNITS_CD,encounter_id,MRN,EVENT_ID,EVENT_NAME,EVENT_MEASURE,charttime,ORDER_ID,RESULT_VAL_num,reftime,fromtime,totime
38,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,230988434efe7ebea0608ee2519d4e19e33b6bda496a46...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-19 02:00:00,NaN,113.0,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17
39,3059679,746116,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,P23052086.0,4593d46b2b798048da32168e3a3fd6d5de61706811b5e4...,Systolic Blood Pressure,Systolic Blood Pressure,2041-01-19 06:00:00,NaN,106.0,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17


In [45]:
si_df_filtered_aki.loc[si_df_filtered_aki.encounter_id=='00b719a07ab022ad6b57b87d79e6ca603914258e457a263811a93a68', :]

,encounter_id,charttime,reftime,fromtime,totime,value,valueUOM
45,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,2041-01-19 02:00:00,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17,NaN,bpm/mmHg
46,00b719a07ab022ad6b57b87d79e6ca603914258e457a26...,2041-01-19 06:00:00,2041-01-19 07:00:17,2041-01-19 01:00:17,2041-01-19 07:00:17,NaN,bpm/mmHg
